# Algorithm Project 2023

## 1. Get data from excel file

In [1]:
import pandas as pd

def get_school_data():
    input_data = pd.read_excel('TestCase/input.xlsx', sheet_name="工作表1", usecols=["校名", "科系", "科系代碼", "欲錄取人數"])
    input_data = input_data.dropna()
    input_data = input_data.values.tolist()

# Convert the department code to string, and the number of students to int
    for i in range(len(input_data)):
        for j in range(2, len(input_data[i])):
            input_data[i][j] = int(input_data[i][j])

    for i in range(len(input_data)):
        input_data[i][2] = str(input_data[i][2])

    return input_data

def get_student_data():
    name = ["准考證號碼", "學生姓名", "第一志願", "正備取01", "第二志願", "正備取02", "第三志願", "正備取03", "第四志願", "正備取04", "第五志願", "正備取05", "第六志願", "正備取06"]

    input_data = pd.read_excel('TestCase/input.xlsx', sheet_name="工作表1", usecols=name)
    input_data = input_data.fillna("None")
    input_data = input_data.values.tolist()


    # Convert '正' to 0, '備n' to n
    for i in range(len(input_data)):
        for j in range(len(input_data[i])):
            # if input_data[i][j] is a string
            if isinstance(input_data[i][j], str):
                temp = input_data[i][j]
                if temp[0] == '正':
                    input_data[i][j] = 0
                elif temp[0] == '備':
                    input_data[i][j] = int(temp[1])
    
    # Convert the student ID to string
    for i in range(len(input_data)):
        input_data[i][0] = str(input_data[i][0])
    
    # convert the department code to string
    for i in range(len(input_data)):
        for j in range(2, len(input_data[i]), 2):
            if input_data[i][j] != "None":
                input_data[i][j] = int(input_data[i][j])
                input_data[i][j] = str(input_data[i][j])
    return input_data

## 2. Gale-Shapley Algorithm

In [2]:
def gale_shapley(schools, students):
    for student in students:
        student.append(2)
    
    school_result = [] # 初始化科系錄取名單
    for i in range(len(schools)):
        school_result.append([])

    student_result = [] # 初始化學生分發結果

    round_count = 1

    waiting_list = students.copy() # 初始化waiting list

    while len(waiting_list) != 0: # 在所有人都分配完之前不斷遊歷students
        round_count += 1

        for student in waiting_list: # 遊歷students

            for i in range(len(schools)): # 尋找第一志願 school
                if schools[i][2] == student[student[14]]:
                    school = schools[i]
                    break

            if len(school_result[i]) == school[3]: # 科系額滿
                if school_result[i][len(school_result[i])-1][school_result[i][len(school_result[i])-1][14]+1] > student[student[14]+1]: # student 比錄取列表的最後一人前面
                    school_result[i][len(school_result[i])-1][14]+=2
                    # ---
                    if school_result[i][len(school_result[i])-1][14] == 14 or school_result[i][len(school_result[i])-1][school_result[i][len(school_result[i])-1][14]] == 'None': # 落榜
                        temp = []
                        temp.append(school_result[i][len(school_result[i])-1][0])
                        temp.append(school_result[i][len(school_result[i])-1][1])
                        temp.append("未錄取")
                        temp.append("未錄取")
                        temp.append("未錄取")
                        student_result.append(temp)
                        students.remove(student)
                    else:
                    # ---
                        students.insert(0, school_result[i][len(school_result[i])-1]) # 將最後一個人繼續分發
                    del school_result[i][len(school_result[i])-1]
                    # school_result[i].append(student)
                    # 依照正備取順序插入school_result
                    for j in range(len(school_result[i])):
                        if student[student[14]+1] < school_result[i][j][school_result[i][j][14]+1]:
                            school_result[i].insert(j, student)
                            break
                        elif j == len(school_result[i])-1:
                            school_result[i].append(student)
                            break
                    students.remove(student)
                else: # student 比錄取列表的最後一人後面
                    student[14]+=2
                    if student[14] == 14 or student[student[14]] == 'None':
                        temp = []
                        temp.append(student[0])
                        temp.append(student[1])
                        temp.append("未錄取")
                        temp.append("未錄取")
                        temp.append("未錄取")
                        student_result.append(temp)
                        students.remove(student)
            else: # 科系還有名額
                if student[student[14]] == 0: # 正取
                    if len(school_result[i]) != 0: # 科系有人
                        if len(school_result[i])+1 <= school[3]: # 加入student後錄取名單沒爆
                            for j in range(len(school_result[i])): # 依照正備取順序插入school_result
                                if student[student[14]+1] < school_result[i][j][school_result[i][j][14]+1]:
                                    school_result[i].insert(j, student)
                                    break
                                elif j == len(school_result[i])-1 and student[student[14]+1] >= school_result[i][j][school_result[i][j][14]+1]:
                                    school_result[i].append(student)
                                    break
                            students.remove(student)
                        else: # 加入student後錄取名單爆了
                            for j in range(len(school_result[i])): # 依照正備取順序插入school_result
                                if student[student[14]+1] < school_result[i][j][school_result[i][j][14]+1]:
                                    school_result[i].insert(j, student)
                                    break
                                elif j == len(school_result[i])-1 and student[student[14]+1] >= school_result[i][j][school_result[i][j][14]+1]:
                                    school_result[i].append(student)
                                    break
                            school_result[i][len(school_result[i])-1][14]+=2
                            # ---
                            if school_result[i][len(school_result[i])-1][14] == 14 or school_result[i][len(school_result[i])-1][school_result[i][len(school_result[i])-1][14]] == 'None': # 落榜
                                temp = []
                                temp.append(school_result[i][len(school_result[i])-1][0])
                                temp.append(school_result[i][len(school_result[i])-1][1])
                                temp.append("未錄取")
                                temp.append("未錄取")
                                temp.append("未錄取")
                                student_result.append(temp)
                                students.remove(student)

                            else:
                            # ---
                                students.insert(0, school_result[i][len(school_result[i])-1]) # 將最後一個人繼續分發
                            del school_result[i][len(school_result[i])-1]
                    else: # 科系沒人
                        school_result[i].append(student) # 直接加入school_result
                        students.remove(student)
                else: # 備取
                    if len(school_result[i]) != 0: # 科系有人
                        if len(school_result[i])+1 <= school[3]: # 加入student後錄取名單沒爆
                            for j in range(len(school_result[i])): # 依照正備取順序插入school_result
                                if student[student[14]+1] < school_result[i][j][school_result[i][j][14]+1]:
                                    school_result[i].insert(j, student)
                                    break
                                elif j == len(school_result[i])-1 and student[student[14]+1] >= school_result[i][j][school_result[i][j][14]+1]:
                                    school_result[i].append(student)
                                    break
                            students.remove(student)
                        else: # 加入student後錄取名單爆了
                            for j in range(len(school_result[i])): # 依照正備取順序插入school_result
                                if student[student[14]+1] < school_result[i][j][school_result[i][j][14]+1]:
                                    school_result[i].insert(j, student)
                                    break
                                elif j == len(school_result[i])-1 and student[student[14]+1] >= school_result[i][j][school_result[i][j][14]+1]:
                                    school_result[i].append(student)
                                    break
                            school_result[i][len(school_result[i])-1][14]+=2
                            # ---
                            if school_result[i][len(school_result[i])-1][14] == 14 or school_result[i][len(school_result[i])-1][school_result[i][len(school_result[i])-1][14]] == 'None': # 落榜
                                temp = []
                                temp.append(school_result[i][len(school_result[i])-1][0])
                                temp.append(school_result[i][len(school_result[i])-1][1])
                                temp.append("未錄取")
                                temp.append("未錄取")
                                temp.append("未錄取")
                                student_result.append(temp)
                                students.remove(student)
                                continue
                            else:
                            # ---
                                students.insert(0, school_result[i][len(school_result[i])-1]) # 將最後一個人繼續分發
                            del school_result[i][len(school_result[i])-1]
                    else: # 科系沒人
                        school_result[i].append(student) # 直接加入school_result
                        students.remove(student)
    
        waiting_list = students.copy()

    for i in range(len(school_result)): # 將科系錄取名單轉換成學生分發結果
        for j in range(len(school_result[i])):
            temp = []
            temp.append(school_result[i][j][0])
            temp.append(school_result[i][j][1])
            temp.append(schools[i][0])
            temp.append(schools[i][1])
            temp.append(schools[i][2])
            student_result.append(temp)

    student_result.sort(key=lambda x: x[0])

    return student_result

## 3. Validate the result

In [3]:
# To verify that the program's results match those obtained through manual calculations

def get_output_data():
    input_data = pd.read_excel('TestCase/output.xlsx', sheet_name="工作表1", usecols=["准考證號碼", "學生姓名", "錄取校名", "錄取科系", "錄取科系代碼"])
    input_data = input_data.dropna()
    input_data = input_data.values.tolist()

# Convert all intergers to strings
    for i in range(len(input_data)):
        for j in range(len(input_data[i])):
            input_data[i][j] = str(input_data[i][j])

    return input_data

def validation(results):
    outputs = get_output_data()
    if outputs == results:
        print("Correct")
    else:
        print("Error")
    return

In [4]:
import time

schools = get_school_data()
students = get_student_data()

t1=time.time()
results = gale_shapley(schools, students)
t2=time.time()

print('准考證號碼 學生姓名 錄取校名 錄取科系 錄取科系代碼')
for result in results:
    print(' '.join(result))

validation(results)
print('Time: ', t2-t1)

准考證號碼 學生姓名 錄取校名 錄取科系 錄取科系代碼
11080002 楊順亦 清華大學 化學系 2004
11080003 劉雅書 台灣大學 醫學系 1001
11080004 郭姵紫 清華大學 化學系 2004
11080005 陳怡喬 清華大學 數學系 2008
11080006 林宜禾 清華大學 數學系 2008
11080007 曹昀志 政治大學 外交系 3009
11080008 李偉昀 政治大學 外交系 3009
11080009 陳雅玲 台灣大學 電機系 1002
11080010 陳建緯 清華大學 數學系 2008
11080011 毛雲誠 台灣大學 電機系 1002
11080012 許俊玲 清華大學 數學系 2008
11080013 王梅智 台灣大學 醫學系 1001
11080014 林佳蓉 未錄取 未錄取 未錄取
Correct
Time:  0.0
